In [1]:
import pandas as pd
import cobra as co

# Convert the Tables that make up gapseq's "full model" to a cobrapy model Object


## Download the relevant tables into this repo's data folder

In [ ]:
!wget -P "data/" "https://raw.githubusercontent.com/jotech/gapseq/f3d74944e5e4ee5a6ab328c4fd46b35fd53cee73/dat/seed_reactions_corrected.tsv"
!wget -P "data/" "https://raw.githubusercontent.com/jotech/gapseq/f3d74944e5e4ee5a6ab328c4fd46b35fd53cee73/dat/seed_metabolites_edited.tsv"
!wget -P "data/" "https://raw.githubusercontent.com/jotech/gapseq/master/dat/seed_Enzyme_Class_Reactions_Aliases_unique_edited.tsv"

## Import the tables as pandas DataFrames

In [2]:
df_reactions = pd.read_csv("data/seed_reactions_corrected.tsv", sep='\t')

In [3]:
df_metabolites = pd.read_csv("data/seed_metabolites_edited.tsv", sep='\t')

In [4]:
df_ec = pd.read_csv("data/seed_Enzyme_Class_Reactions_Aliases_unique_edited.tsv", sep='\t')

In [5]:
df_rxns_corrected = df_reactions[df_reactions["gapseq.status"]=="corrected"]

## Introduce MIRIAM compliant headers to annotation columns

In [6]:
df_metabolites = df_metabolites.rename(
    columns={"biocycID": "biocyc", 
             "biggID": "bigg.metabolite",
             "keggID": "kegg.compound",
             "InChI": "inchi",
             "chebiID": "chebi",
             "reactomeID": "reactome",
             "hmdbID": "hmdb",
             "InChIKey": "inchikey",
             "MNX_ID": "metanetx.chemical"
            }
    )

df_metabolites

,id,metanetx.chemical,abbreviation,name,formula,mass,source,charge,is_core,is_obsolete,...,aliases,smiles,inchikey,hmdb,reactome,chebi,inchi,kegg.compound,bigg.metabolite,biocyc
0,cpd00001,MNXM2,h2o,H2O,H2O,1.800000e+01,ModelSEED,0,1,0,...,NaN,O,XLYOFNOQVPJJNP-UHFFFAOYSA-N,HMDB01039;HMDB02111,109276;113518;113519;113521;141343;1605715;189...,15377,InChI=1S/H2O/h1H2,C00001;C01328,h2o,WATER
1,cpd00002,MNXM3,atp,ATP,C10H13N5O13P3,5.040000e+02,ModelSEED,-3,1,0,...,NaN,O[C@@H]1[C@@H](COP(=O)(OP(=O)(OP(=O)(O)[O-])[O...,ZKHQWZAMYRWXGA-KQYNXXCUSA-J,HMDB00538,113592;113593;211579;29358;389573;5632460;5696...,57299,InChI=1S/C10H16N5O13P3/c11-8-5-9(13-2-12-8)15(...,C00002,atp,ATP
2,cpd00003,MNXM8,nad,NAD,C21H26N7O14P2,6.620000e+02,ModelSEED,-1,1,0,...,NaN,O[C@@H]1[C@H](O)[C@H](O[C@H]1[n+]1cccc(c1)C(=O...,BAWFJGJZGIEFAR-NNYOXOHSSA-M,HMDB00902,113526;192307;194653;29360;352330,57540,InChI=1S/C21H27N7O14P2/c22-17-12-19(25-7-24-17...,C00003,nad,NAD
3,cpd00004,MNXM10,nadh,NADH,C21H27N7O14P2,6.630000e+02,ModelSEED,-2,1,0,...,NaN,O[C@@H]1[C@H](O)[C@H](O[C@H]1N1C=CCC(=C1)C(=O)...,BOPGDPNILDQYTO-NNYOXOHSSA-L,HMDB01487,192305;194697;29362;73473,57945,InChI=1S/C21H29N7O14P2/c22-17-12-19(25-7-24-17...,C00004,nadh,NADH
4,cpd00005,MNXM6,nadph,NADPH,C21H26N7O17P3,7.420000e+02,ModelSEED,-4,1,0,...,NaN,O[C@@H]1[C@H](O)[C@H](O[C@H]1N1C=CCC(=C1)C(=O)...,ACFIXJIJDZMPPO-NNYOXOHSSA-J,HMDB00221;HMDB00799;HMDB06341,113600;113601;113602;194725;2000347;29364;3516...,57783,InChI=1S/C21H30N7O17P3/c22-17-12-19(25-7-24-17...,C00005,nadph,NADPH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27728,cpd90015,MNXM102616,"2-[(3,4-dihydroxyphenyl)methyl]-2,4,6-trihydro...",Alphitonin,C15H12O7,3.042516e+02,CHEBI,0,0,0,...,NaN,Oc1cc2OC(O)(Cc3ccc(O)c(O)c3)C(=O)c2c(O)c1,VCLACNNZBMRRES-UHFFFAOYSA-N,NaN,NaN,69013,InChI=1S/C15H12O7/c16-8-4-11(19)13-12(5-8)22-1...,NaN,NaN,NaN
27729,cpd90016,MNXM118330,6'-Hydroxy-O-desmethylangolensin,6'-Hydroxy-O-desmethylangolensin,C15H14O5,2.742687e+02,CHEBI,0,0,0,...,NaN,CC(C(=O)c1c(O)cc(O)cc1O)c1ccc(O)cc1,IEFUAUZFJJOQMC-UHFFFAOYSA-N,HMDB0041695,NaN,88662,InChI=1S/C15H14O5/c1-8(9-2-4-10(16)5-3-9)15(20...,NaN,NaN,NaN
27730,cpd90017,MNXM147296,Ser_Thr_residue,Protein-Linked Serine Or Threonine Residue (O-...,HOR,1.000000e+07,VMH,0,0,0,...,NaN,NaN,NaN,NaN,NaN,53604,NaN,NaN,Ser_Thr,NaN
27731,cpd90018,NaN,(R)-dihydrodaidzein,"(R)-4',7-dihydroxyisoflavanone",C15H12O4,2.562600e+02,MetaCyc,0,0,0,...,NaN,C3(\C([C@H]1(C(C2(\C(\OC1)=C/C(/O)=C\C=2))=O))...,JHYXBPPMXZIHKG-ZDUSSCGKSA-N,NaN,NaN,136678,InChI=1S/C15H12O4/c16-10-3-1-9(2-4-10)13-8-19-...,NaN,NaN,CPD-24413


## Build the cobrapy model object from scratch

In [7]:
universal = co.Model()
universal.id = "universal"
universal.name = "Gap-Seq-derived Universal Model"

### Build a list of reaction objects from info in the table and the reaction equation strings

In [8]:
df_rxns_corrected = df_rxns_corrected.rename(
    columns={"ec_numbers": "ec-code"
            }
    )

In [9]:
def handle_annotation(entry):
    # Could be a list delimited by ';', a single string, an int or float, or NaN. 
    # Here we try to handle each case
    if pd.isnull(entry):
        return ''
    if type(entry) != str:
        return str(entry)
    if ";" in entry:
        return entry.split(";")
    return str(entry)

In [10]:
reactions = []

annotation_columns = [
    "ec-code"
]

notes_columns = [
    'abbreviation',
    'code',
    'stoichiometry',
    'is_transport',
    'equation',
    'definition',
    'reversibility',
    'direction',
    'abstract_reaction',
    'pathways',
    'aliases',
    'deltag',
    'deltagerr',
    'compound_ids',
    'status',
    'is_obsolete',
    'linked_reaction',
    'notes',
    'is_copy_of',
    'gapseq.status'
]

def convert_equation_to_bigg_compartments(equation):
        equation = equation.replace("[0]", "_c").replace("[1]", "_e").replace("[2]", "_p")
        equation = equation.replace("(", "").replace(")", "")
        return str(equation)
    

def define_rxn(row):
    # A function to define the reaction objects based on the row of the corrected Reaction DataFrame
    rxn = co.Reaction(
        id = str(row["id"]),
        name = str(row["name"]),
    )
    rxn.annotation.update({k : handle_annotation(row[k]) for k in annotation_columns if handle_annotation(row[k]) is not None})
    rxn.notes.update({k : handle_annotation(row[k]) for k in notes_columns if handle_annotation(row[k]) is not None})
    rxn.annotation["seed.reaction"] = row["id"]
    
    universal.add_reactions([rxn])
    
    rxn.build_reaction_from_string(convert_equation_to_bigg_compartments(row.equation))

for index, row in df_rxns_corrected.iterrows():
    define_rxn(row)

unknown metabolite 'cpd00001_c' created
unknown metabolite 'cpd00012_c' created
unknown metabolite 'cpd00009_c' created
unknown metabolite 'cpd00067_c' created
unknown metabolite 'cpd00011_c' created
unknown metabolite 'cpd03049_c' created
unknown metabolite 'cpd00020_c' created
unknown metabolite 'cpd00056_c' created
unknown metabolite 'cpd00871_c' created
unknown metabolite 'cpd00169_c' created
unknown metabolite 'cpd00007_c' created
unknown metabolite 'cpd01433_c' created
unknown metabolite 'cpd00075_c' created
unknown metabolite 'cpd00178_c' created
unknown metabolite 'cpd00109_c' created
unknown metabolite 'cpd00268_c' created
unknown metabolite 'cpd00110_c' created
unknown metabolite 'cpd01414_c' created
unknown metabolite 'cpd10515_c' created
unknown metabolite 'cpd10516_c' created
unknown metabolite 'cpd00003_c' created
unknown metabolite 'cpd00423_c' created
unknown metabolite 'cpd00004_c' created
unknown metabolite 'cpd00730_c' created
unknown metabolite 'cpd00005_c' created


unknown metabolite 'cpd00191_c' created
unknown metabolite 'cpd00308_c' created
unknown metabolite 'cpd03409_c' created
unknown metabolite 'cpd00347_c' created
unknown metabolite 'cpd00113_c' created
unknown metabolite 'cpd00202_c' created
unknown metabolite 'cpd00283_c' created
unknown metabolite 'cpd00907_c' created
unknown metabolite 'cpd00206_c' created
unknown metabolite 'cpd00654_c' created
unknown metabolite 'cpd00207_c' created
unknown metabolite 'cpd03194_c' created
unknown metabolite 'cpd00208_c' created
unknown metabolite 'cpd00210_c' created
unknown metabolite 'cpd00460_c' created
unknown metabolite 'cpd00213_c' created
unknown metabolite 'cpd00860_c' created
unknown metabolite 'cpd00693_c' created
unknown metabolite 'cpd00843_c' created
unknown metabolite 'cpd00482_c' created
unknown metabolite 'cpd00448_c' created
unknown metabolite 'cpd00226_c' created
unknown metabolite 'cpd00227_c' created
unknown metabolite 'cpd00346_c' created
unknown metabolite 'cpd03733_c' created


unknown metabolite 'cpd03366_c' created
unknown metabolite 'cpd03398_c' created
unknown metabolite 'cpd03397_c' created
unknown metabolite 'cpd03495_c' created
unknown metabolite 'cpd03496_c' created
unknown metabolite 'cpd03817_c' created
unknown metabolite 'cpd03818_c' created
unknown metabolite 'cpd03819_c' created
unknown metabolite 'cpd03915_c' created
unknown metabolite 'cpd03916_c' created
unknown metabolite 'cpd03919_c' created
unknown metabolite 'cpd03920_c' created
unknown metabolite 'cpd00166_c' created
unknown metabolite 'cpd00149_c' created
unknown metabolite 'cpd03913_c' created
unknown metabolite 'cpd03914_c' created
unknown metabolite 'cpd00561_c' created
unknown metabolite 'cpd04073_c' created
unknown metabolite 'cpd00728_c' created
unknown metabolite 'cpd07946_c' created
unknown metabolite 'cpd01557_c' created
unknown metabolite 'cpd03043_c' created
unknown metabolite 'cpd08288_c' created
unknown metabolite 'cpd08301_c' created
unknown metabolite 'cpd08316_c' created


unknown metabolite 'cpd00061_e' created
unknown metabolite 'cpd00220_e' created
unknown metabolite 'cpd01030_e' created
unknown metabolite 'cpd03698_c' created
unknown metabolite 'cpd00048_e' created
unknown metabolite 'cpd00048_c' created
unknown metabolite 'cpd00264_c' created
unknown metabolite 'cpd00264_e' created
unknown metabolite 'cpd00036_e' created
unknown metabolite 'cpd00076_e' created
unknown metabolite 'cpd00118_e' created
unknown metabolite 'cpd00378_e' created
unknown metabolite 'cpd00209_c' created
unknown metabolite 'cpd11451_c' created
unknown metabolite 'cpd11606_c' created
unknown metabolite 'cpd11640_c' created
unknown metabolite 'cpd11798_c' created
unknown metabolite 'cpd11796_c' created
unknown metabolite 'cpd11795_c' created
unknown metabolite 'cpd11797_c' created
unknown metabolite 'cpd00074_c' created
unknown metabolite 'cpd01311_c' created
unknown metabolite 'cpd00104_c' created
unknown metabolite 'cpd03091_c' created
unknown metabolite 'cpd11932_c' created


unknown metabolite 'cpd15551_c' created
unknown metabolite 'cpd00224_c' created
unknown metabolite 'cpd00224_e' created
unknown metabolite 'cpd00051_e' created
unknown metabolite 'cpd15500_c' created
unknown metabolite 'cpd15499_c' created
unknown metabolite 'cpd00041_e' created
unknown metabolite 'cpd15421_c' created
unknown metabolite 'cpd00149_e' created
unknown metabolite 'cpd00266_c' created
unknown metabolite 'cpd15441_c' created
unknown metabolite 'cpd08305_c' created
unknown metabolite 'cpd15440_c' created
unknown metabolite 'cpd15306_c' created
unknown metabolite 'cpd15307_c' created
unknown metabolite 'cpd15522_c' created
unknown metabolite 'cpd15308_c' created
unknown metabolite 'cpd15309_c' created
unknown metabolite 'cpd15310_c' created
unknown metabolite 'cpd15311_c' created
unknown metabolite 'cpd15312_c' created
unknown metabolite 'cpd15417_c' created
unknown metabolite 'cpd15423_c' created
unknown metabolite 'cpd15422_c' created
unknown metabolite 'cpd15419_c' created


unknown metabolite 'cpd15897_c' created
unknown metabolite 'cpd15898_c' created
unknown metabolite 'cpd15891_c' created
unknown metabolite 'cpd15894_c' created
unknown metabolite 'cpd15892_c' created
unknown metabolite 'cpd15879_c' created
unknown metabolite 'cpd20413_c' created
unknown metabolite 'cpd20412_c' created
unknown metabolite 'cpd20411_c' created
unknown metabolite 'cpd15869_c' created
unknown metabolite 'cpd15870_c' created
unknown metabolite 'cpd15871_c' created
unknown metabolite 'cpd15872_c' created
unknown metabolite 'cpd00289_c' created
unknown metabolite 'cpd15881_c' created
unknown metabolite 'cpd03523_c' created
unknown metabolite 'cpd15851_c' created
unknown metabolite 'cpd15847_c' created
unknown metabolite 'cpd00703_c' created
unknown metabolite 'cpd00703_e' created
unknown metabolite 'cpd00278_c' created
unknown metabolite 'cpd11175_c' created
unknown metabolite 'cpd00143_c' created
unknown metabolite 'cpd00452_c' created
unknown metabolite 'cpd00868_c' created


unknown metabolite 'cpd16788_c' created
unknown metabolite 'cpd16790_c' created
unknown metabolite 'cpd16811_c' created
unknown metabolite 'cpd00167_c' created
unknown metabolite 'cpd16812_c' created
unknown metabolite 'cpd16831_c' created
unknown metabolite 'cpd16850_c' created
unknown metabolite 'cpd16849_c' created
unknown metabolite 'cpd02339_c' created
unknown metabolite 'cpd03671_c' created
unknown metabolite 'cpd12458_e' created
unknown metabolite 'cpd00132_e' created
unknown metabolite 'cpd00046_e' created
unknown metabolite 'cpd00052_e' created
unknown metabolite 'cpd00053_e' created
unknown metabolite 'cpd00119_e' created
unknown metabolite 'cpd00322_e' created
unknown metabolite 'cpd00107_e' created
unknown metabolite 'cpd00558_e' created
unknown metabolite 'cpd00558_c' created
unknown metabolite 'cpd03047_c' created
unknown metabolite 'cpd00069_e' created
unknown metabolite 'cpd00091_e' created
unknown metabolite 'cpd00062_e' created
unknown metabolite 'cpd16910_c' created


unknown metabolite 'cpd22200_e' created
unknown metabolite 'cpd22200_c' created
unknown metabolite 'cpd22431_e' created
unknown metabolite 'cpd22431_c' created
unknown metabolite 'cpd01157_e' created
unknown metabolite 'cpd01157_c' created
unknown metabolite 'cpd28726_e' created
unknown metabolite 'cpd28726_c' created
unknown metabolite 'cpd22633_e' created
unknown metabolite 'cpd22633_c' created
unknown metabolite 'cpd24378_e' created
unknown metabolite 'cpd24378_c' created
unknown metabolite 'cpd09394_e' created
unknown metabolite 'cpd09394_c' created
unknown metabolite 'cpd24451_e' created
unknown metabolite 'cpd24451_c' created
unknown metabolite 'cpd24452_e' created
unknown metabolite 'cpd24452_c' created
unknown metabolite 'cpd00265_e' created
unknown metabolite 'cpd00265_c' created
unknown metabolite 'cpd25500_e' created
unknown metabolite 'cpd25500_c' created
unknown metabolite 'cpd25730_e' created
unknown metabolite 'cpd25730_c' created
unknown metabolite 'cpd25949_e' created


### Build a list of metabolite objects

In [11]:
annotation_columns = [
    "biocyc", 
    "bigg.metabolite", 
    "kegg.compound", 
    "inchi", 
    "chebi", 
    "reactome", 
    "hmdb", 
    "inchikey", 
    "metanetx.chemical"
]

notes_columns = [
    'abbreviation',
    'mass',
    'source',
    'is_core',
    'is_obsolete',
    'linked_compound',
    'is_cofactor',
    'deltag',
    'deltagerr',
    'pka',
    'pkb',
    'abstract_compound',
    'comprised_of',
    'aliases'
]

for index, row in df_metabolites.iterrows():
    # Identify which metabolites have already been added and supplement all additional
    # info from the metabolite DataFrame
    metabolites = universal.metabolites.query(row["id"])
    for met in metabolites:
        met.name = row["name"]
        met.formula = handle_annotation(row["formula"])
        met.charge = row["charge"]
        met.compartment = met.id.split("_")[1]
        met.annotation.update({k : handle_annotation(row[k]) for k in annotation_columns if handle_annotation(row[k]) is not None})
        met.notes.update({k : handle_annotation(row[k]) for k in notes_columns if handle_annotation(row[k]) is not None})
        met.annotation["seed.compound"]=row["id"]


### Add EC number annotation to reactions

In [12]:
for index, row in df_ec.iterrows():
    reaction_strings = []
    if "|" in row["MS ID"]:
        reaction_strings = row["MS ID"].split("|")
    else:
        reaction_strings.append(row["MS ID"])
    
    for r in reaction_strings:
        reaction_object = universal.reactions.query(r)
        if reaction_object:        
            reaction_object[0].annotation["ec-code"] = str(row["External ID"])

### Write the model as SBML

In [13]:
import logging
logging.getLogger().setLevel(logging.DEBUG)

In [14]:
co.io.write_sbml_model(universal, "universal.sbml")

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00001 "diphosphate phosphohydrolase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00011 "pyruvate:thiamin diphosphate acetaldehydetransferase (decarboxylating)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00018 "3-phospho-D-glycerate carboxy-lyase (dimerizing; D-ribulose-1,5-bisphosphate-forming)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00157 "Acetyl-CoA:formate C-acetyltransferase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00183 "L-Glutamate 5-semialdehyde:NAD+ oxidoreductase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00202 "acetyl-CoA:2-oxoglutarate C-acetyltransferase (thioester-hydrolysing, carboxymethyl forming)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00391 "riboflavin-5-phosphate phosphohydrolase (acid optimum)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00399 "L-Arginine,NADPH:oxygen oxidoreductase (nitric-oxide-forming)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00410 "UTP:ammonia ligase (ADP-forming)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situatio

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00541 "L-threonine acetaldehyde-lyase (glycine-forming)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00549 "D-Fructose-1,6-bisphosphate 1-phosphohydrolase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00550 "beta-D-fructose-2,6-bisphosphate 2-phosphohydrolase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation 

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00690 "Formate:tetrahydrofolate ligase (ADP-forming)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00693 "5-Methyltetrahydrofolate:L-homocysteine S-methyltransferase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn00703 "alpha-D-Glucose-1-phosphate:alpha-D-glucose-1-phosphate 4-alpha-D-glucosyltransferase (dephosphorylating)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contex

...

LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn37809 "3-hydroxyadipyl-CoA dehydrogenase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn37873 "1.1.1.8-RXN.e">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn37888 "1.2.1.13-RXN.e">>.
LibSBML error code -3: The requ

LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn38488 "3-oxo-dodecanoyl-[acyl-carrier protein] reductase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn38489 "3-oxo-dodecanoyl-[acyl-carrier protein] reductase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn38490

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn38993 "Succinic dehydrogenase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn39043 "tetrapeptide L,D-carboxypeptidase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn39044 "N-Acetyl-D-glucosamine(anhydrous)N-Acetylmuramyl-tetrapeptide amidase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not mak

Error encountered trying to <Setting notes on sbase: <Reaction R_rxn90069 "7-beta hydroxysteroid dehydrogenase, NAD+ dependent, ursocholate specific">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn90073 "12-alpha hydroxysteroid dehydrogenase, NAD+ dependent">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn90074 "12beta-hydroxysteroid:NADP+ 12-oxidoreductase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a

LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn90094 "nitric oxide reductase (cytochrome c)">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn90095 "nitrous-oxide reductase">>.
LibSBML error code -3: The requested action could not be performed. This can occur in a variety of contexts, such as passing a null object as a parameter in a situation where it does not make sense to permit a null object.
Error encountered trying to <Setting notes on sbase: <Reaction R_rxn90099 "2-(4-Hydroxyphenyl)-propionate perme

# Test with Memote

In [ ]:
!memote report snapshot "universal.sbml"